In [3]:
import geopandas as gpd
from shapely.geometry import Point
import spectral as sp
import folium as fol
import pystac_client
import planetary_computer
import rioxarray as rxr
from rioxarray.merge import merge_arrays
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import stackstac
import os

ModuleNotFoundError: No module named 'planetary_computer'

In [4]:
os.listdir()

NameError: name 'os' is not defined

In [5]:
SBB_data_file = '/home/jovyan/shared-public/AVIRIS-NG data NASA UW Hackweek FMI Project/AVIRIS-NG-March_2021/SNEX21_SSR_ang20210319t093125_SBB_rfl_v2z1.img'
SBB_header_file = '/home/jovyan/shared-public/AVIRIS-NG data NASA UW Hackweek FMI Project/AVIRIS-NG-March_2021/SNEX21_SSR_ang20210319t093125_SBB_rfl_v2z1.hdr'

In [6]:
!gdaltindex -t_srs EPSG:4326 index.gpkg SNEX21_SSR_ang20210319t093125_SBB_rfl_v2z1*.img*

ERROR 4: SNEX21_SSR_ang20210319t093125_SBB_rfl_v2z1*.img*: No such file or directory
Warning 1: Unable to open SNEX21_SSR_ang20210319t093125_SBB_rfl_v2z1*.img*, skipping.


In [7]:
%pwd
home_folder = %pwd
SBB_data_file = f'{home_folder}/{SBB_data_file}'
SBB_header_file = f'{home_folder}/{SBB_header_file}'
image = sp.open_image(SBB_header_file)

FileNotFoundError: Unable to locate file "/home/jovyan/shared-public/NASA-UW-Hackweek-2024-FMI-Project//home/jovyan/shared-public/AVIRIS-NG data NASA UW Hackweek FMI Project/AVIRIS-NG-March_2021/SNEX21_SSR_ang20210319t093125_SBB_rfl_v2z1.hdr". If the file exists, use its full path or place its directory in the SPECTRAL_DATA environment variable.

In [12]:
# Coordinates
longitude = -107.71113019999999949
latitude = 37.90715906999999874

# Create a GeoDataFrame with the point geometry
gdf = gpd.GeoDataFrame(geometry=[Point(longitude, latitude)])

# Specify the coordinate reference system (CRS)
gdf.set_crs(epsg=4326, inplace=True)  # EPSG:4326 is the standard for WGS84

# Save the GeoDataFrame to a GeoJSON file
output_file = "SBB_point.geojson"
gdf.to_file(output_file, driver="GeoJSON")

print(f"GeoJSON file '{output_file}' created successfully!")

GeoJSON file 'SBB_point.geojson' created successfully!


In [13]:
roi = gpd.read_file('SBB_point.geojson')

In [14]:
sbb = gpd.read_file('https://raw.githubusercontent.com/snowex-hackweek/tutorial-data/main/SnowEx-2022/AVIRIS-NG/SBB_basin.geojson')
swamp_angel = gpd.read_file('https://raw.githubusercontent.com/snowex-hackweek/tutorial-data/main/SnowEx-2022/AVIRIS-NG/SwampAngel.geojson')

In [17]:
# Create a map with multiple layers to explore where the lines are
## Layer 1 used as base layer

sbb_layer = sbb.explore(
    name='SBB basin',
    color='rgba(0, 128, 0, 0.5)'  # RGBA format where alpha=0.5 for 50% transparency
)

roi.explore(
    m=sbb_layer,  # Add this layer to the sbb_layer map
    name = 'TARTES location',
    color = 'red'
)
## Layer 2
swamp_angel.explore(
    m=sbb_layer,                   
    name='Swamp Angel Study Plot'
)

# Top right box to toggle layer visibility
fol.LayerControl().add_to(sbb_layer)

# Show the final map with all layer
sbb_layer

In [20]:
import folium

# Layer 1: SBB basin
sbb_layer = sbb.explore(
    name='SBB basin',
    color='rgba(0, 128, 0, 0.5)'  # RGBA format where alpha=0.5 for 50% transparency
)

# Layer 2: TARTES location
roi.explore(
    m=sbb_layer,  # Add this layer to the sbb_layer map
    name='TARTES Point 1 location',
    color='red'
)

# Layer 3: Swamp Angel Study Plot
swamp_angel.explore(
    m=sbb_layer,                   
    name='Swamp Angel Study Plot'
)

# Add layer control
folium.LayerControl().add_to(sbb_layer)

# Create a custom legend
legend_html = '''
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 200px;
    height: 150px;
    background-color: white;
    border:2px solid grey;
    z-index:9999;
    font-size:14px;
    ">
    <h4 style="margin: 10px;">Legend</h4>
    <i style="background:rgba(0, 128, 0, 0.5);width:18px;height:18px;float:left;margin-right:8px;"></i>SBB basin<br>
    <i style="background:red;width:18px;height:18px;float:left;margin-right:8px;"></i>TARTES location<br>
    <i style="background:blue;width:18px;height:18px;float:left;margin-right:8px;"></i>Swamp Angel Study Plot
</div>
'''

# Add the legend to the map
sbb_layer.get_root().html.add_child(folium.Element(legend_html))

# Show the final map with all layers and the legend
sbb_layer


In [34]:
import folium

# Create a base map with satellite tiles
base_map = folium.Map(location=[sbb.geometry.centroid.y.mean(), sbb.geometry.centroid.x.mean()],
                      zoom_start=10, tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}', attr='Google Satellite')
# Layer 1: SBB basin
sbb_layer = sbb.explore(
    m=base_map,  # Attach to the base map with satellite tiles
    name='Senator Beck Basin',
    color='rgba(0, 148, 0, 0.5)'  # RGBA format where alpha=0.5 for 50% transparency
)

# Layer 2: TARTES location
roi.explore(
    m=sbb_layer,  # Add this layer to the sbb_layer map
    name='TARTES/Field spectroscopy Point 1 location',
    color='red'
)

# Layer 3: Swamp Angel Study Plot
swamp_angel.explore(
    m=sbb_layer,                   
    name='Swamp Angel Study Plot'
)

# Add layer control
folium.LayerControl().add_to(sbb_layer)

# Create a custom legend
legend_html = '''
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 350px;
    height: 100px;
    background-color: white;
    border:2px solid grey;
    z-index:9999;
    font-size:14px;
    ">
    <h4 style="margin: 10px;">Legend</h4>
    <i style="background:rgba(0, 128, 0, 0.5);width:18px;height:18px;float:left;margin-right:8px;"></i>Senator Beck Basin<br>
    <i style="background:red;width:18px;height:18px;float:left;margin-right:8px;"></i>TARTES/Field spectroscopy Point 1 location<br>
    <i style="background:blue;width:18px;height:18px;float:left;margin-right:8px;"></i>Swamp Angel Study Plot
</div>
'''

# Add the legend to the map
sbb_layer.get_root().html.add_child(folium.Element(legend_html))

# Show the final map with all layers, satellite background, and the legend
sbb_layer



/tmp/ipykernel_569/1841117043.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  base_map = folium.Map(location=[sbb.geometry.centroid.y.mean(), sbb.geometry.centroid.x.mean()],


In [20]:
import rasterio

with rasterio.open(SBB_data_file) as sbb_subset:
    x, y = sbb_subset.index(roi.geometry.x, roi.geometry.y) # The index methods returns arrays
    print(x[0], y[0])

1006595 203719


In [21]:
image = sp.open_image(SBB_header_file)
print(image)

	Data Source:   '/home/jovyan/shared-public/AVIRIS-NG data NASA UW Hackweek FMI Project/AVIRIS-NG-March_2021/SNEX21_SSR_ang20210319t093125_SBB_rfl_v2z1.img'
	# Rows:           1666
	# Samples:         634
	# Bands:           425
	Interleave:        BIL
	Quantization:  32 bits
	Data format:   float32


In [22]:
image_data = image.load()

In [23]:
#plot spectrum for a given pixel
my_pixel = image_data[x[0], y[0]] 
plt.plot (bands, my_pixel, color='blue')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Reflectance')
plt.show()

IndexError: index 1006595 is out of bounds for axis 0 with size 1666